In [ ]:
# IMPORTS
from ipywidgets import HBox, VBox, Label, Layout, AppLayout, Output
from IPython.display import display
import sys, os
import yaml

# append parent of cwd to sys path to find modules
root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

# surpress pygame version prompt
os.environ['PYGAME_HIDE_SUPPORT_PROMPT'] = "hide"

# enable matplotlib widgets
%matplotlib widget

from src.app_batch import AppBatch as App
from src.datatypes.call_response_set import CallResponseSet
from dashboards.widgets import inputHeading, inputSelect, inputUpload, generationHeading, checkpointSelect, tempSlider, generationInfo, adaptationHeading, stepsSelect, startButton, applyGenerationSettingsButton, applyAdaptationSettingsButton, generationAmount, adaptationAmount
from dashboards.widgets import h2_heading 

from src.io.plot import multitrack_pianoroll, multi_evaluation_bars
from src.io.output import play_button
from src.io.input import loadMidiFile

In [ ]:
##########################
#C#    OUTPUT & APP    ###
##########################

log = Output()
output = Output()

pianoroll_container = Output()
play_button_container = Output()
evaluation_container = Output()
meta_l_container = Output()
meta_m_container = Output()
meta_r_container = Output()
generation_info = Output()

# init
app = App(log=log)

with generation_info:
    display(generationInfo)

In [ ]:
##########################
#M#   SELECTION DATA   ###
##########################

adaptation_operations = app.get_adaptation_operations()
generators = app.get_generators()
demo_melodies = app.get_demo_input()

checkpointSelect.options = [(g[0], generators.index(g)) for g in generators]
stepsSelect.options = adaptation_operations
inputSelect.options = [(d[0], demo_melodies.index(d)) for d in demo_melodies]

In [ ]:
##########################
#C#     CONTROLLER     ###
##########################

def apply_generator_settings(b):
    generator = app.apply_generator_settings(checkpointSelect.value, tempSlider.value)
    generation_info.clear_output()
    with generation_info:
        print('Using: ' + generator)

def apply_adaptation_settings(b):
    app.apply_adaptation_settings(stepsSelect.value)

def run_adaptation(b):
    # reset output containers
    pianoroll_container.clear_output()
    play_button_container.clear_output()
    evaluation_container.clear_output()
    meta_l_container.clear_output()
    meta_m_container.clear_output()
    meta_r_container.clear_output()

    # TODO handle file upload
    input_path = demo_melodies[inputSelect.value][1]
    cr_set = app.run(input_path)

    # create pianoroll plots from results and print them
    midi_list = [cr_set.input_sequence.sequence, cr_set.generated_base_sequence.sequence, cr_set.output_sequence.sequence]
    name_list = ['Input', 'Generation', 'Result']
    multitrack_pianoroll(midi_list, name_list, pianoroll_container)

    # display play buttons for audio
    with play_button_container: 
        display(h2_heading('Input:'))
    play_button(midi_list[0], play_button_container)
    with play_button_container: 
        display(h2_heading('Generation:'))    
    play_button(midi_list[1], play_button_container)
    with play_button_container: 
        display(h2_heading('Adaptation:'))
    play_button(midi_list[2], play_button_container)

    # create evaluation plots
    eval_data = [cr_set.generation_similarity['normalized'], cr_set.output_similarity['normalized']]
    multi_evaluation_bars(eval_data, evaluation_container, ['Generated Sequence', 'Adapted Sequence'])

    # print meta data
    with meta_l_container:
        print('Input Analysis:\n', yaml.dump(cr_set.input_sequence.analysis, sort_keys=False, default_flow_style=False))
    with meta_m_container:
        print('Gen Analysis:\n', yaml.dump(cr_set.output_sequence.analysis, sort_keys=False, default_flow_style=False))
    with meta_r_container:
        print('Adaptation Meta:\n', yaml.dump(cr_set.output_sequence.meta, sort_keys=False, default_flow_style=False))


startButton.on_click(run_adaptation)
applyGenerationSettingsButton.on_click(apply_generator_settings)
applyAdaptationSettingsButton.on_click(apply_adaptation_settings)

In [ ]:
##########################
#V#  SETTINGS SIDEBAR  ###
##########################

side_bar_layout = Layout(border='1px solid grey', padding='10px')

settings_box = VBox([
        generationHeading,
        HBox([Label('Model & Checkpoint:'), checkpointSelect]),
        HBox([Label('Temperature:'), tempSlider]),
        generation_info,
        applyGenerationSettingsButton,
        adaptationHeading,
        HBox([Label('Steps:'), stepsSelect]),
        applyAdaptationSettingsButton,
        inputHeading,
        inputSelect,
        HBox([Label('Or upload own file:'), inputUpload]),
        HBox([Label('Amount of generations:'), generationAmount]),
        HBox([Label('Amount of adaptations per generation:'), adaptationAmount]),
        startButton,
    ], layout = side_bar_layout)

In [ ]:
##########################
#V#    RESULTS AREA    ###
##########################

main_layout = Layout(border='1px solid grey', padding='10px')

pianorolls = HBox([pianoroll_container, play_button_container])
meta = HBox([meta_l_container, meta_m_container, meta_r_container])

results_box = VBox([pianorolls, evaluation_container, meta, output], layout=main_layout)

In [ ]:
##########################
#V#     APP LAYOUT     ###
##########################

AppLayout(left_sidebar=settings_box,
          center=results_box,
          header=None,
          footer=log,
          right_sidebar=None,
          pane_widths=[1,2,0],
          pane_heights=[0,4,1])

In [ ]:
# run_adaptation(startButton)